**Simone Rittenhouse**

**Assignment 3**

Loading in Database:

In [ ]:
gdrivePath="/content/drive/MyDrive/Data Management and Analytics/"
import sqlite3
import pandas as pd

In [ ]:
conn = sqlite3.connect(gdrivePath+"chinook.db")

In [ ]:
def execSQL(conn,query):
  conn.execute(query)
  conn.commit()

In [ ]:
def allrowsSelect(conn,query):
  cursor = conn.execute(query)
  for row in cursor:
   print(row)

Checking tables:

In [ ]:
pd.read_sql_query("select name from sqlite_master where type='table' or type='view' order by name",conn)

,name
0,albums
1,artists
2,customers
3,employees
4,genres
5,invoice_items
6,invoices
7,media_types
8,playlist_track
9,playlists


##**Questions:**

In the invoices table, which BillingState in the USA has the most invoices? (Note that the invoices table has orders from many countries.)

In [ ]:
q = '''
SELECT BillingState, COUNT(invoiceID) as numInvoices
FROM
(SELECT invoiceId, BillingState
FROM invoices
WHERE BillingCountry = 'USA')
GROUP BY BillingState
ORDER BY numInvoices DESC
'''

pd.read_sql_query(q, conn)

,BillingState,numInvoices
0,CA,21
1,AZ,7
2,FL,7
3,IL,7
4,MA,7
5,NV,7
6,NY,7
7,TX,7
8,UT,7
9,WA,7


Oddly, all of the customers except for one bought 38 tracks. How many tracks did that one customer who bought a different number buy? (Note you can ignore the Quantity field in the invoice_items table which is always one.)

In [ ]:
q = '''
SELECT COUNT(invoiceLineId) as numTracks
FROM customers JOIN invoices USING(customerId) JOIN invoice_items USING(invoiceId)
GROUP BY customerId
HAVING numTracks <> 38
'''

pd.read_sql_query(q, conn)

,numTracks
0,36


How many albums have Classical tracks?

In [ ]:
q = '''
SELECT COUNT(DISTINCT albumId) as numAlbums
FROM albums JOIN tracks USING(albumid) JOIN genres USING(genreId)
GROUP BY genres.name
HAVING genres.name = 'Classical'
'''

pd.read_sql_query(q, conn)

,numAlbums
0,72


How many Classical tracks were purchased in total? (Note you can ignore the Quantity field in the invoice_items table which is always one.) 

In [ ]:
q = '''
SELECT COUNT(invoiceLineId) as classicalCount
FROM invoice_items JOIN tracks USING(trackId) JOIN genres USING(genreId)
GROUP BY genres.name
HAVING genres.name = 'Classical'
'''

pd.read_sql_query(q, conn)

,classicalCount
0,41


What is the name of the customer who purchased the most Classical tracks?

In [ ]:
q = '''
SELECT firstName, lastName, COUNT(invoicelineId) as mostClassical
FROM customers JOIN invoices USING(customerId) JOIN invoice_items USING(invoiceId) JOIN tracks USING(trackId) JOIN genres USING(genreID)
WHERE genres.name = 'Classical'
GROUP BY customerID
ORDER BY mostClassical DESC
'''

pd.read_sql_query(q, conn)

,FirstName,LastName,mostClassical
0,Isabelle,Mercier,6
1,Bjørn,Hansen,5
2,François,Tremblay,4
3,Fernanda,Ramos,4
4,Frank,Ralston,4
5,Patrick,Gray,4
6,Luís,Gonçalves,2
7,Astrid,Gruber,2
8,Camille,Bernard,2
9,Marc,Dubois,2


On the Brazilian Music playlist, what is the TrackID of the track that comes first alphabetically?

In [ ]:
q = '''
SELECT trackId, tracks.name
FROM playlists JOIN playlist_track USING(playlistId) JOIN tracks USING(trackId)
WHERE playlists.name = 'Brazilian Music'
ORDER BY tracks.name
LIMIT 1
'''

pd.read_sql_query(q, conn)

,TrackId,Name
0,236,A Banda


What is the name of the album that is responsible for the most revenue, in terms of the sales that are shown in the invoice_items table? (Note you can ignore the Quantity field in the invoice_items table which is always one.) 

In [ ]:
q = '''
SELECT title, SUM(revenue) as total_revenue FROM
(SELECT title, SUM(invoice_items.unitPrice) as revenue
FROM albums JOIN tracks USING(albumId) JOIN invoice_items USING(trackId)
GROUP BY invoiceId
ORDER BY revenue DESC)
GROUP BY title
ORDER BY total_revenue DESC
LIMIT 1
'''

pd.read_sql_query(q, conn)

,title,total_revenue
0,"Battlestar Galactica (Classic), Season 1",51.71


In [ ]:
q = '''
SELECT title, SUM(invoice_items.unitPrice) as revenue
FROM albums JOIN tracks USING(albumId) JOIN invoice_items USING(trackId)
GROUP BY title
ORDER BY revenue DESC
LIMIT 1
'''

pd.read_sql_query(q, conn)

,Title,revenue
0,"Battlestar Galactica (Classic), Season 1",35.82


What is the name of the support rep with the most customers? Note that SupportRepid in the Customers table corresponds to Employeeid in the Employees table.

In [ ]:
q = '''
SELECT e.firstName, e.lastName, SUM(customerId) as numCustomers
FROM employees as e JOIN customers ON e.employeeId = customers.supportrepid
GROUP BY employeeid
ORDER BY numCustomers DESC
'''

pd.read_sql_query(q, conn)

,FirstName,LastName,numCustomers
0,Jane,Peacock,701
1,Steve,Johnson,546
2,Margaret,Park,523


What genre accounts for the second-highest sales?

In [ ]:
q = '''
SELECT Name, MIN(totalSpent) as secondHighest
FROM
(SELECT genres.Name as name, SUM(invoices.total) as totalSpent
FROM genres JOIN tracks USING(genreID) JOIN invoice_items USING(trackId) JOIN invoices USING(invoiceID)
GROUP BY genreId
ORDER BY totalSpent DESC
LIMIT 2)
'''

pd.read_sql_query(q, conn)

,name,secondHighest
0,Latin,3472.55


What artist has the most albums in the database? 

In [ ]:
q = '''
SELECT artists.name, COUNT(title) as numAlbums
FROM artists JOIN albums USING(artistId)
GROUP BY artists.name
ORDER BY numAlbums DESC, artists.name
LIMIT 1
'''

pd.read_sql_query(q, conn)

,Name,numAlbums
0,Iron Maiden,21


Find the names of the tracks on the album "Back to Black" and put them in alphabetical order. What is the name of the third track (in alphabetical order, not in the order in which they appeared on the album)>

In [ ]:
q = '''
SELECT tracks.name
FROM albums JOIN tracks USING(albumId)
WHERE albums.title = 'Back to Black'
ORDER BY tracks.name
'''

pd.read_sql_query(q, conn)

,Name
0,Back to Black
1,He Can Only Hold Her
2,Just Friends
3,Love Is a Losing Game
4,Me & Mr. Jones
5,Rehab
6,Rehab (Hot Chip Remix)
7,Some Unholy War
8,Tears Dry On Their Own
9,Wake Up Alone


Find the name and artistid of the artists in the database who do not have albums in the database. What is the name of the artist in this group with the lowest artistid?

In [ ]:
q = '''
SELECT name, artistId
FROM artists LEFT JOIN albums USING(artistId)
WHERE albumId IS NULL
ORDER BY artistId
'''

pd.read_sql_query(q, conn)

,Name,ArtistId
0,Milton Nascimento & Bebeto,25
1,Azymuth,26
2,João Gilberto,28
3,Bebel Gilberto,29
4,Jorge Vercilo,30
...,...,...
66,DJ Dolores & Orchestra Santa Massa,192
67,Seu Jorge,193
68,Sabotage E Instituto,194
69,Stereo Maracana,195


What is the name of the third longest album, in terms of the total milliseconds of its tracks?

In [ ]:
q = '''
SELECT title, SUM(milliseconds) as albumLength
FROM albums JOIN tracks USING(albumId)
GROUP BY albumId
ORDER BY albumLength DESC
'''

pd.read_sql_query(q, conn)

,Title,albumLength
0,"Lost, Season 3",70665582
1,"Battlestar Galactica (Classic), Season 1",70213784
2,"Lost, Season 1",64854936
3,"Lost, Season 2",63289631
4,"Heroes, Season 1",59780268
...,...,...
342,Handel: Music for the Royal Fireworks (Origina...,120000
343,"Charpentier: Divertissements, Airs & Concerts",110266
344,SCRIABIN: Vers la flamme,101293
345,Monteverdi: L'Orfeo,66639


What billing city in what billing country, outside the USA and Europe, had the largest number of invoices? Note most of the billing countries are in Europe: the ones that aren't (other than the USA) are Canada, Australia, Chile, India, Brazil, and Argentina.

In [ ]:
q = '''
SELECT count(invoiceId) as numInvoice, BillingCity, BillingCountry
FROM invoices
WHERE BillingCountry = 'Canada' OR BillingCountry = 'Australia' OR BillingCountry = 'Chile' OR BillingCountry = 'India'
OR BillingCountry = 'Brazil' OR BillingCountry = 'Argentina'
GROUP BY BillingCity
ORDER BY numInvoice DESC
'''

pd.read_sql_query(q, conn)

,numInvoice,BillingCity,BillingCountry
0,14,São Paulo,Brazil
1,7,Brasília,Brazil
2,7,Buenos Aires,Argentina
3,7,Delhi,India
4,7,Edmonton,Canada
5,7,Halifax,Canada
6,7,Montréal,Canada
7,7,Ottawa,Canada
8,7,Rio de Janeiro,Brazil
9,7,Santiago,Chile
